<a href="https://colab.research.google.com/github/rulas99/thermal_anomaly_detection/blob/main/ChooseAndSplitData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
from glob import glob
import numpy as np
import cv2
from tqdm import tqdm
from pandas import DataFrame
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [151]:
label_imgs = glob('/content/drive/MyDrive/segment_sentinel2_hotspot/mask_correction/*')

In [119]:
def check_label(path):
  img = cv2.imread(path)
  return f'{path}-yes' if 255 in img else f'{path}-no'

In [108]:
res = list(map(check_label,label_imgs))

In [120]:
with Pool(4) as p:
  res = p.map(check_label, label_imgs)

In [121]:
data = list(zip(*[i.split('-') for i in res]))

In [166]:
df = DataFrame({'label_path':data[0],'anomaly':data[1]})

In [167]:
df.to_csv('/content/drive/MyDrive/segment_sentinel2_hotspot/images_info.csv',index=False)

In [168]:
t_df = df[df.anomaly=='yes']

In [169]:
t_df.shape

(509, 2)

In [170]:
train, test = train_test_split(t_df, test_size=.2, train_size=.8, random_state=99)

In [171]:
len(train), len(test)

(407, 102)

In [172]:
train_names = [f'{i.split("/")[-1][:-5]}.png' for i in train.label_path]
test_names = [f'{i.split("/")[-1][:-5]}.png' for i in test.label_path]

In [173]:
thermal_imgs_path = '/content/drive/MyDrive/segment_sentinel2_hotspot/classify_sentinel2_trainingdataset'

In [174]:
train['ori_path'] = [f'{thermal_imgs_path}/{i}' for i in train_names]
test['ori_path'] = [f'{thermal_imgs_path}/{i}' for i in test_names]

In [177]:
train.to_csv('/content/drive/MyDrive/segment_sentinel2_hotspot/train.csv',index=False)
test.to_csv('/content/drive/MyDrive/segment_sentinel2_hotspot/test.csv',index=False)